In [7]:
from typing import List, Set
from enum import Enum
from random import randint

In [8]:
def eq(n1: float | int, n2: float | int, tol: float = 0.5e-3) -> bool:
    """ Проверка двух чисел с плавающей запятой на равенство с точностью до заданного десятичного знака. """
    if abs(n1 - n2) <= tol:
        return True
    return False


def gt(n1: float, n2: float, tol: float = 0.5e-3) -> bool:
    """ Возвращает True, если n1 больше либо равно n2 с точностью до tol. """
    return n1 - n2 >= -tol


def ls(n1: float, n2: float, tol: float = 0.5e-3) -> bool:
    """ Возвращает True, если n1 меньше либо равно n2 с точностью до tol. """
    return gt(n2, n1, tol)


In [9]:
class Payload:
    def __init__(self, x: float) -> None:
        self.x: float = x
        self.trackNumber: int = 1
        self.iplEdge: Edge = Edge()
    
    def __repr__(self) -> str:
        return f"{{x: {self.x}, li: {self.trackNumber % 10_000}, br: {self.trackNumber // 10_000}}}"


class Node: 
    def __init__(self, x: float, lineIndex: int = 1):
        self.x: float = x
        self.trackNumber: int = lineIndex
    
    def __repr__(self) -> str:
        return f"{{x: {self.x}, li: {self.trackNumber}}}"


class Edge:
    def __init__(self) -> None:
        self.c: complex = 0+0j

    def getSourceNode(self) -> Node:
        return Node(randint(0, 100))
    
    def getTargetNode(self) -> Node:
        return Node(randint(0, 100))


class Side(Enum):
    Left = 0
    Right = 1


class AcNetworkLattice:
    def cond(
        self,
        lineIndex1: int,
        side1: Side,
        lineIndex2: int,
        side2: Side,
        length: float
    ) -> complex:
        return 0 + 0j
    

class CircuitGraph:
    def addEdge(self, n1: Node, n2: Node, e: Edge):
        pass


class Cell:
    """ Ячейка ТС. """

    def __init__(
            self,
            xLeft: float,
            xRight: float,
            leftNodes: List[Node],
            rightNodes: List[Node],
            zeroNode: Node,
            label: str,
            lattice: AcNetworkLattice
    ):
        self.next: "Cell" | None = None
        self.prev: "Cell" | None = None
        self.xLeft: float = xLeft
        self.xRight: float = xRight
        self.leftNodes = leftNodes
        self.rightNodes = rightNodes
        self.__zeroNode = zeroNode
        self.label: str = label
        self.lattice = lattice
        self.edges: Set[Edge] = set()

    def pullRightSection(self, destPoint: float) -> None:
        """ Притянуть ячейку за правое сечение в заданную точку. """
        if not eq(destPoint, self.xRight):
            destPoint = round(destPoint, 3)
            self.xRight = destPoint
            for n in self.rightNodes:
                n.x = destPoint
            if destPoint < self.xLeft:
                self.xLeft = destPoint
                for n in self.leftNodes:
                    n.x = destPoint
                if self.prev is not None:
                    self.prev.pullRightSection(destPoint)
            self.__updateEdgeConductivities()
            if self.next is not None:
                self.next.pullLeftSection(destPoint)

    def pullLeftSection(self, destPoint: float) -> None:
        """ Притянуть ячейку за левое сечение в заданную точку. """
        if not eq(destPoint, self.xLeft):
            destPoint = round(destPoint, 3)
            self.xLeft = destPoint
            for n in self.leftNodes:
                n.x = destPoint
            if destPoint > self.xRight:
                self.xRight = destPoint
                for n in self.rightNodes:
                    n.x = destPoint
                if self.next is not None:
                    self.next.pullLeftSection(destPoint)
            self.__updateEdgeConductivities()
            if self.prev is not None:
                self.prev.pullRightSection(destPoint)
    
    def findNodeToConnect(self, pl: Payload) -> Node:
        for lst in self.leftNodes, self.rightNodes:
            for n in lst:
                if eq(n.x, pl.x) and pl.trackNumber == n.trackNumber:
                    return n
        raise Exception("Не найден узел для подключения нагрузки")
    
    def __updateEdgeConductivities(self):
        length = self.xRight - self.xLeft
        for e in self.edges:
            n1 = e.getSourceNode()
            n2 = e.getTargetNode()
            side1 = Side.Left if n1 in self.leftNodes else Side.Right
            side2 = Side.Left if n2 in self.leftNodes else Side.Right
            if n2 == self.__zeroNode:
                e.c = self.lattice.cond(n1.trackNumber, side1, n1.trackNumber, side1, length)
            elif n1.trackNumber == self.__zeroNode:
                e.c = self.lattice.cond(n2.trackNumber, side2, n2.trackNumber, side2, length)
            e.c = self.lattice.cond(n1.trackNumber, side1, n2.trackNumber, side2, length)

    def __repr__(self) -> str:
        return f"{{label: {self.label}: left: {{x: {self.xLeft}, nodes: {self.leftNodes}}}, right: {{x: {self.xRight}, nodes: {self.rightNodes}}}}}"

In [10]:
from typing import Tuple


class Partition:
    def __init__(self, xLeft: float, xRight: float, leftNodes: List[Node], rightNodes: List[Node], zeroNode: Node, lattice: AcNetworkLattice) -> None:
        self.xLeft = round(xLeft, 3)
        self.xRight = round(xRight, 3)
        self.leftNodes = leftNodes
        self.rightNodes = rightNodes
        self.zeroNode: Node = zeroNode
        self.firstCell = None
        self.lastCell = None
        self.__payloads: List[Payload] = []
        self.__cellQty = 0
        self.lattice = lattice

    def ensureCapacity(self, payloadCoordinates: List[float]):
        coordinates: Set[float] = set()
        for x in payloadCoordinates:
            if gt(x, self.xLeft) and ls(x, self.xRight) and not eq(x, self.xLeft) and not eq(x, self.xRight):
                coordinates.add(round(x, 3))
        print(coordinates)
        if len(coordinates) > self.__cellQty:
            self.__cellQty = len(coordinates) + 1

    def initCells(self) -> None:
        self.firstCell = Cell(self.xLeft, self.xRight, self.leftNodes, self.rightNodes, self.zeroNode, "1", self.lattice)
        prev = self.firstCell
        for i in range(1, self.__cellQty):
            rightNodes = [Node(self.xRight, n.trackNumber) for n in self.rightNodes]
            next = Cell(self.xRight, self.xRight, prev.rightNodes, rightNodes, self.zeroNode, str(i + 1), self.lattice)
            next.prev = prev
            prev.next = next
            prev = next
        self.lastCell = prev

    def addPayload(self, pl:Payload) -> bool:
        if self.firstCell is None or self.lastCell is None:
            raise Exception()
        if gt(pl.x, self.xLeft) and ls(pl.x, self.xRight):
            self.__payloads.append(pl)
            return True
        return False

    def addPayloads(self, pls: List[Payload]) -> int:
        cnt = 0
        for pl in pls:
            if self.addPayload(pl):
                cnt += 1
        return cnt

    def arrangePayloads(self, graph: CircuitGraph) -> None:
        if self.firstCell is None or self.lastCell is None:
            raise Exception()
        self.__payloads.sort(key=lambda pl: pl.x)
        self.firstCell.pullRightSection(self.lastCell.xRight) # стянуть все ячейки к правой границе
        cell = self.firstCell
        for pl in self.__payloads:
            cell, n = self.__addPayloadToCell(cell, pl)
            graph.addEdge(n, self.zeroNode, pl.iplEdge)
    
    def removePayloads(self):
        self.__payloads.clear()   

    def __addPayloadToCell(self, cell: Cell, pl: Payload) -> Tuple[Cell, Node]:
        if eq(cell.xLeft, pl.x) or eq(cell.xRight, pl.x):
            return cell, cell.findNodeToConnect(pl)
        elif pl.x < cell.xRight:
            cell.pullRightSection(pl.x)
            return cell, cell.findNodeToConnect(pl)
        elif pl.x > cell.xRight and cell.next is not None:
            return self.__addPayloadToCell(cell.next, pl)
        else:
            raise Exception()

In [11]:
payloads = [Payload(x) for x in (-0.0003, 11, 19.5, 17.9996, 18, 18.5, 3)]
print(f"payloads = {payloads}")

leftNodes = [Node(0, i + 1) for i in range(3)]
rightNodes = [Node(20, i + 1) for i in range(3)]
p = Partition(0, 20, leftNodes, rightNodes, Node(0), AcNetworkLattice())
p.ensureCapacity([p.x for p in payloads])
p.initCells()
p.addPayloads(payloads)

p.arrangePayloads(CircuitGraph())

cell = p.firstCell
cellSummary = []
while cell is not None:
    cellSummary.append(f"{cell.xLeft}..{cell.xRight}")
    cell = cell.next
print("cells = " + " | ".join(cellSummary))

payloads = [{x: -0.0003, li: 1, br: 0}, {x: 11, li: 1, br: 0}, {x: 19.5, li: 1, br: 0}, {x: 17.9996, li: 1, br: 0}, {x: 18, li: 1, br: 0}, {x: 18.5, li: 1, br: 0}, {x: 3, li: 1, br: 0}]
{3, 11, 18.5, 19.5, 18.0}
cells = 0..3 | 3..11 | 11..18.0 | 18.0..18.5 | 18.5..19.5 | 19.5..20
